In [98]:
import pandas as pd
import numpy as np

In [99]:
df_raw_data = pd.read_json('../0_datasets/regression/imoveis.json', orient='columns')
df_raw_data.head()

,ident,listing
0,"{'customerID': '775564-BOJSMVON', 'source': 'W...","{'types': {'unit': 'Outros', 'usage': 'Residen..."
1,"{'customerID': '660895-AUENKNYY', 'source': 'W...","{'types': {'unit': 'Apartamento', 'usage': 'Re..."
2,"{'customerID': '751522-JESYFEQL', 'source': 'W...","{'types': {'unit': 'Outros', 'usage': 'Residen..."
3,"{'customerID': '714052-GAAEWYKS', 'source': 'W...","{'types': {'unit': 'Outros', 'usage': 'Comerci..."
4,"{'customerID': '568886-ZIBFOMCC', 'source': 'W...","{'types': {'unit': 'Apartamento', 'usage': 'Re..."


In [100]:
dados_lista = pd.json_normalize(df_raw_data.ident)
dados_lista_2 = pd.json_normalize(df_raw_data.listing, sep='_')
df_data = pd.concat([dados_lista, dados_lista_2], axis=1)

# FILTRO
filtro_residencial = (df_data['types_usage'] == 'Residencial') & (df_data['address_city'] == 'Rio de Janeiro')
df_data_residencial = df_data[filtro_residencial]

# LIMPEZA
df_data_residencial.reset_index(drop=True, inplace=True)

# FIXING DATATYPES
df_data_residencial = df_data_residencial.astype({
    "prices_price": "float64",
    "prices_tax_iptu": "float64",
    "prices_tax_condo": "float64",
    "features_usableAreas": "int64",
    "features_totalAreas": "int64"
})

df_data_residencial.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69588 entries, 0 to 69587
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   customerID                69588 non-null  object 
 1   source                    69588 non-null  object 
 2   types_unit                69588 non-null  object 
 3   types_usage               69588 non-null  object 
 4   address_city              69588 non-null  object 
 5   address_location_lon      69449 non-null  float64
 6   address_location_lat      69449 non-null  float64
 7   address_zone              69588 non-null  object 
 8   address_neighborhood      69588 non-null  object 
 9   prices_price              69588 non-null  float64
 10  prices_tax_iptu           60863 non-null  float64
 11  prices_tax_condo          63527 non-null  float64
 12  features_bedrooms         69588 non-null  int64  
 13  features_bathrooms        69588 non-null  int64  
 14  featur

In [101]:
df_data_residencial['address_zone'] = df_data_residencial['address_zone'].replace('', np.nan)

In [102]:
df_data_residencial.address_zone.isnull().sum()

149

In [103]:
dict_zones = df_data_residencial[~df_data_residencial['address_zone'].isna()].drop_duplicates(subset=['address_neighborhood', 'address_zone']).to_dict('records')
dict_zones = {item['address_neighborhood']: item['address_zone'] for item in dict_zones}

for bairro, zona in dict_zones.items():
    df_data_residencial.loc[(df_data_residencial['address_neighborhood'] == bairro) & (df_data_residencial['address_zone'].isna()), 'address_zone'] = zona
df_data_residencial.address_zone.isnull().sum()

df_data_residencial.prices_tax_iptu.fillna(0, inplace=True)
df_data_residencial.prices_tax_condo.fillna(0, inplace=True)

df_data_residencial.drop(['customerID', 'source', 'types_usage', 'address_city', 'address_location_lon', 'address_location_lat', 'address_neighborhood'], inplace=True, axis=1)

columns_remap = {
    "types_unit": 'unit', 'address_zone': 'zone', 'prices_price': 'price', 
    'prices_tax_condo': 'tax_condo', 'prices_tax_iptu': 'tax_iptu',  'features_bedrooms': 'bedrooms', 
    'features_bathrooms': 'bathrooms', 'features_suites': 'suites', 'features_parkingSpaces': 'parkingSpaces',
    'features_usableAreas': 'usableAreas', 'features_totalAreas': 'totalAreas', 'features_floors': 'floors',
    'features_unitsOnTheFloor': 'unitsOnTheFloor', 'features_unitFloor': 'unitFloor'
}

df_data_residencial.rename(columns=columns_remap, inplace=True)

/tmp/ipykernel_325500/1131446016.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_data_residencial.prices_tax_iptu.fillna(0, inplace=True)
/tmp/ipykernel_325500/1131446016.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inpla

In [104]:
df_data_residencial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69588 entries, 0 to 69587
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   unit             69588 non-null  object 
 1   zone             69588 non-null  object 
 2   price            69588 non-null  float64
 3   tax_iptu         69588 non-null  float64
 4   tax_condo        69588 non-null  float64
 5   bedrooms         69588 non-null  int64  
 6   bathrooms        69588 non-null  int64  
 7   suites           69588 non-null  int64  
 8   parkingSpaces    69588 non-null  int64  
 9   usableAreas      69588 non-null  int64  
 10  totalAreas       69588 non-null  int64  
 11  floors           69588 non-null  int64  
 12  unitsOnTheFloor  69588 non-null  int64  
 13  unitFloor        69588 non-null  int64  
dtypes: float64(3), int64(9), object(2)
memory usage: 7.4+ MB


In [105]:

df_data_residencial.head()

,unit,zone,price,tax_iptu,tax_condo,bedrooms,bathrooms,suites,parkingSpaces,usableAreas,totalAreas,floors,unitsOnTheFloor,unitFloor
0,Outros,Zona Oeste,45000.0,0.0,150.0,0,0,0,1,62,62,0,0,0
1,Apartamento,Zona Oeste,45000.0,0.0,120.0,2,1,0,1,44,0,0,0,0
2,Outros,Zona Oeste,50000.0,0.0,100.0,0,0,0,0,132,132,0,0,0
3,Apartamento,Zona Oeste,50000.0,120.0,400.0,3,2,1,1,60,0,3,4,2
4,Outros,Zona Oeste,50000.0,0.0,0.0,0,0,0,0,200,200,0,0,0


In [106]:
correlacao = df_data_residencial.corr()
correlacao

ValueError: could not convert string to float: 'Outros'